In [2]:
from pianoq.misc.mplc_writeup_imports import * 
from pianoq.misc.mplc_lab_imports import * 

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_10_08_find_z10'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

In [ ]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1 

dz10s = np.linspace(-15e-3, 15e-3, 11)

for dz10 in dz10s:
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    masks_path =    fr'{dir_path}\{timestamp}_U1U_dz10={dz10}.masks'
    mplc_res_path = fr'{dir_path}\{timestamp}_U1U_dz10={dz10}.mplcsim'
    mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path, dead_middle_zone=0, last_plane_extra_dist=dz10)
    mplc_sim.res.saveto(mplc_res_path, smaller=True)

In [3]:
import re 
masks_paths = glob.glob(rf'{dir_path}\*.masks')
all_ds = np.array([re.findall('.*dz10=(.*).masks', path)[0] for path in masks_paths]).astype(float)

In [9]:
mplc = MPLCDevice()

backlash = 0
wait_after_move = 0.2
mxs, mys = get_signal_motors(backlash=backlash, wait_after_move=wait_after_move)

integration_time = 1
coin_window = 2e-9
tt = get_timetagger(coin_window=coin_window, integration_time=integration_time)

Got Zaber motors
God timetagger


In [11]:
# Signal scans - zaber motors are faster, and these are reflected. But maybe I should actually look at the detector that is not reflected, since I will not move it, so I want to find what is best for it, and then move the mirror accordingly... 
for i in range(len(all_ds)):
    masks = get_good_masks(masks_path=masks_paths[i], modes_to_keep=modes_to_keep, phases_path=None)
    mplc.load_masks(masks)
    mplc.restore_location()
    
    scan_saveto_path = rf'{dir_path}\{tnow()}_2spots_fine__dz10={all_ds[i]}.scan'

    start_x = 12.7
    end_x = 13.15
    start_y = 5.5
    end_y = 6.35
    
    resolution = 1 
    x_pixels = 18
    y_pixels = 34
    pixel_size_x = 0.025 
    pixel_size_y = 0.025 
    
    scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path)   
    _ = scanner.scan(x_motor=mxs, y_motor=mys, ph=tt)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 3374. Singles2: 936. Coincidence: 0.00.
dur: 1. pix: 0, 1. Singles1: 3319. Singles2: 1006. Coincidence: 0.00.
dur: 3. pix: 0, 2. Singles1: 3331. Singles2: 936. Coincidence: 0.00.
dur: 4. pix: 0, 3. Singles1: 3332. Singles2: 957. Coincidence: 0.00.
dur: 6. pix: 0, 4. Singles1: 3355. Singles2: 1056. Coincidence: 0.00.
dur: 7. pix: 0, 5. Singles1: 3370. Singles2: 1011. Coincidence: 0.00.
dur: 9. pix: 0, 6. Singles1: 3337. Singles2: 1069. Coincidence: 0.00.
dur: 10. pix: 0, 7. Singles1: 3283. Singles2: 1067. Coincidence: 0.00.
dur: 12. pix: 0, 8. Singles1: 3314. Singles2: 1022. Coincidence: 0.00.
dur: 13. pix: 0, 9. Singles1: 3373. Singles2: 1092. Coincidence: 0.00.
dur: 15. pix: 0, 10. Singles1: 3327. Singles2: 1029. Coincidence: 0.00.
dur: 16. pix: 0, 11. Singles1: 3223. Singles2: 1069. Coincidence: 0.00.
dur: 17. pix: 0, 12. Singles1: 3232. Singles2: 1088. Coincidence: 0.00.
dur: 19. pix: 0, 13. Singles1: 3180. Si

In [24]:
mxs.move_absolute(12.94)
mys.move_absolute(5.84)
print(mxs.get_position())
print(mys.get_position())

12.93999825
5.840015625


In [25]:
backlash = 0
wait_after_move = 0.2
mxi, myi = get_idler_motors(backlash=backlash, wait_after_move=wait_after_move)

Got Thorlabs motors


In [ ]:
# Idler scans - I don't want to move detectors, so the best z here will be the one I aim for also for the second detector by changing the mirror location / angle 
phases_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_19_14_12_fine_new_mirror_from_zero_more.phases"
for i in range(len(all_ds)):
    masks = get_good_masks(masks_path=masks_paths[i], modes_to_keep=modes_to_keep, phases_path=phases_path)
    mplc.load_masks(masks)
    mplc.restore_location()
    
    scan_saveto_path = rf'{dir_path}\{tnow()}_2spots_fine_idler_dz10={all_ds[i]}.scan'

    start_x = 9.15
    end_x = 9.55
    start_y = 4.25
    end_y = 5
    
    resolution = 1 
    x_pixels = 16
    y_pixels = 30
    pixel_size_x = 0.025 
    pixel_size_y = 0.025 
    
    scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path)   
    _ = scanner.scan(x_motor=mxi, y_motor=myi, ph=tt)

[0.         0.         5.21490009 0.         0.         0.
 0.         0.59253797 0.         0.         0.         0.
 1.41333115 0.         0.         0.         0.         1.02995202
 0.         0.         0.         0.         5.50020255 0.
 0.         0.         0.         0.33918276 0.         0.
 0.         0.         6.19378008 0.         0.         0.
 0.         0.10820319 0.         0.         0.         0.
 0.46709363 0.         0.         0.         0.         6.00202645
 0.         0.        ]
Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 4232. Singles2: 7990. Coincidence: 0.00.
dur: 2. pix: 0, 1. Singles1: 4452. Singles2: 7998. Coincidence: 0.00.
dur: 4. pix: 0, 2. Singles1: 5322. Singles2: 8153. Coincidence: 1.00.
dur: 5. pix: 0, 3. Singles1: 6348. Singles2: 8048. Coincidence: 0.00.
dur: 7. pix: 0, 4. Singles1: 7395. Singles2: 7941. Coincidence: 0.00.
dur: 9. pix: 0, 5. Singles1: 8677. Singles2: 7985. Coincidence: 1.00.
dur: 10. pix: 0, 6. Sin

In [ ]:
# Finally, the really important thing to check is the correlation! 
# The signal is in a good location, and I will search for coincidences in the corresponding idlerignal spot (thorlabs are slower, but I will have a large integration time anyway...)
phases_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_19_14_12_fine_new_mirror_from_zero_more.phases"

integration_time = 10
coin_window = 2e-9
tt = get_timetagger(coin_window=coin_window, integration_time=integration_time)

for i in range(len(all_ds)):
    masks = get_good_masks(masks_path=masks_paths[i], modes_to_keep=modes_to_keep, phases_path=phases_path)
    mplc.load_masks(masks)
    mplc.restore_location()
    
    scan_saveto_path = rf'{dir_path}\{tnow()}_fine_corrs_dz10={all_ds[i]}.scan'

    start_x = 9.15
    end_x =   9.55
    start_y = 4.6
    end_y = 5
    
    resolution = 1 
    x_pixels = 16
    y_pixels = 16
    pixel_size_x = 0.025 
    pixel_size_y = 0.025 
    
    scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path)   
    _ = scanner.scan(x_motor=mxi, y_motor=myi, ph=tt)